# 回帰モデルによる因果推論

## 前提

- d分離後に残った変数(共変量)に対して適用
- 線形回帰モデルを用いるので，因果の効果や変数間の関係が線形であること(現実世界は非線形であることが多い)
- 構造方程式モデルを直接求める方法(なのかな？)

## 実装テーマ

<img src="./images/model_example.jpg" width=500>

#### 今回は②にあたる，『テレビCMを観た』と『商品を購入』の間に，どれだけの大きさの因果関係が存在するのかを推論する

- 今回のサンプル数$i$は200とする
- 性別は$x_1$とし，一様分布によって『男性』を"0"，『女性』を"1"とする
- 年齢は$x_2$とし，一様分布によって15歳〜75歳までを生成する

- テレビCMを観た$Z^i$は，観た時を"1"，観なかった時を"0"とする
- テレビCMを観た$Z^i$は，年齢$x_1$が高く，性別$x_2$が女性("1")の方が，観る確率が高いとする
- $Z^i$はシグモイド関数，
$$ \mathrm{sigmoid}(x) =  \frac{1}{1 + \mathrm{e}^{- \alpha t}},$$
によって計算する
- この時$t$は，
$$ t = 10x_1 + x_2 - 40 + \mathrm{noise}^i_t ,$$
とする
- $\alpha$は0.1とする

- 商品を購入$Y^i$は，
\begin{eqnarray}
    Y^i &=& 30(1 - x^i_1) - x^i_2 + 10Z^i + 80 + \mathrm{noise}^i_y \nonumber， \\
        &=& 30 - 30x^i_1 - x^i_2 + 10Z^i + 80 + \mathrm{noise}^i_y \nonumber， \\
        &=& -30x^i_1 - x^i_2 + 10Z^i + 110 + \mathrm{noise}^i_y \nonumber，
\end{eqnarray}
によって決まるとする
- つまり，男性の方が購入量が高く，年齢が高くなるにつれ購入量が減る
- テレビCMを観た$Z^i$の係数は"10"なので，<u>テレビCMによる購入量への効果は+10が正解となる</u>
- それぞれの係数(因果の大きさ)は，$w_1=-30$，$w_2=-1$，$w_3=10$，$w_4=110$となる

## 実装

In [26]:
import numpy as np
import pandas as pd
from sklearn.linear_model import LinearRegression

### ダミーデータの生成

In [27]:
np.random.seed(seed=1234)

# サンプル数は200
num_data = 200
# 性別(0-1の離散一様分布)
x_1 = np.random.randint(0, 2, num_data)
# 年齢(15-75の離散一様分布)
x_2 = np.random.randint(15, 76, num_data)
# テレビCMを観たかどうか
noise_t = np.random.randn(num_data) # 標準正規分布に従う乱数
t = 10*x_1 + x_2 - 40 + noise_t
z = 1 / (1 + np.exp(-0.1*t))
Z = np.where(z < 0.5, 0, 1) # 0.5より小さいものは"0"，以上は"1"

# 商品の購入量(目的変数)の算出
noise_y = np.random.randn(num_data)
Y = 30 * (1-x_1) - x_2 + 10*Z + 80 + noise_y

In [31]:
df = pd.DataFrame({"性別":x_1,"年齢":x_2,"CMを観た":Z,"購入量":Y})
df.head()

,性別,年齢,CMを観た,購入量
0,1,27,0,51.563426
1,1,20,0,60.210717
2,0,68,1,51.103096
3,1,32,1,56.275607
4,0,31,0,80.792339


### 平均による因果推論(間違い)

In [35]:
print(df[df['CMを観た']==0].mean())
print('------------------------')
print(df[df['CMを観た']==1].mean())

性別        0.367647
年齢       26.705882
CMを観た     0.000000
購入量      72.291045
dtype: float64
------------------------
性別        0.537879
年齢       55.537879
CMを観た     1.000000
購入量      48.200746
dtype: float64


CMを観ると購入量が減る → なんか変じゃない？？？  
つまり，その上流にある変数の存在(交絡因子)を考慮できていない．

### 回帰モデルによる因果推論

In [37]:
X = df[['性別','年齢','CMを観た']]
y = df['購入量']

reg = LinearRegression().fit(X,y)

print("係数：{}".format(reg.coef_))

係数：[-29.97564711  -1.00283133   9.92613613]


この結果から，『CMを観た』の因果の大きさは"9.92613613"ということがわかった．  
事前に設定した値は"10"なので，正しく推論できていることが確認された．